<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_192WinLen__APR28_WeightedLoss_NoVolume_Debayan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

Cloning into 'idl-spring-22-project-deepar'...
remote: Enumerating objects: 1869, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1869 (delta 19), reused 0 (delta 0), pack-reused 1829
Receiving objects: 100% (1869/1869), 83.04 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (896/896), done.
Checking out files: 100% (1951/1951), done.


In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

In [3]:
import pickle
with open('/content/idl-spring-22-project-deepar/data/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open('/content/idl-spring-22-project-deepar/data/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [4]:
print(index_to_company)
print(company_to_index)

{0: 'A', 1: 'AA', 2: 'AAL', 3: 'ABBV', 4: 'ABNB', 5: 'ABT', 6: 'ABUS', 7: 'ACAD', 8: 'ACFN', 9: 'ACI', 10: 'ACN', 11: 'ADBE', 12: 'AEHR', 13: 'AEO', 14: 'AEZS', 15: 'AFMD', 16: 'AG', 17: 'AGQ', 18: 'AGR', 19: 'AHT', 20: 'AI', 21: 'AJRD', 22: 'ALDX', 23: 'ALK', 24: 'ALPP', 25: 'ALXN', 26: 'AMAT', 27: 'AMBA', 28: 'AMC', 29: 'AMD', 30: 'AMGN', 31: 'AMKR', 32: 'AMP', 33: 'AMRN', 34: 'AMRS', 35: 'AMT', 36: 'AMZN', 37: 'ANET', 38: 'ANY', 39: 'APA', 40: 'APPS', 41: 'APRN', 42: 'APT', 43: 'AR', 44: 'ARCB', 45: 'ARKK', 46: 'ARWR', 47: 'ASAN', 48: 'ASM', 49: 'ASML', 50: 'ASO', 51: 'ASTR', 52: 'ATER', 53: 'ATIP', 54: 'ATOM', 55: 'ATUS', 56: 'ATVI', 57: 'ATVK', 58: 'AU', 59: 'AUD', 60: 'AUNXF', 61: 'AUPH', 62: 'AVCT', 63: 'AVGO', 64: 'AVLR', 65: 'AZ', 66: 'B', 67: 'BA', 68: 'BABA', 69: 'BABL', 70: 'BAC', 71: 'BB', 72: 'BBBY', 73: 'BBIG', 74: 'BBIO', 75: 'BBY', 76: 'BEEM', 77: 'BFLY', 78: 'BHP', 79: 'BHVN', 80: 'BIDU', 81: 'BIIB', 82: 'BILL', 83: 'BKKT', 84: 'BKNG', 85: 'BLK', 86: 'BLNK', 87: 'BLUE

In [5]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'



stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [ ]:
NUM_TRAIN_SAMPLES = 500000
NUM_TEST_SAMPLES  = 100000

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [ ]:
np.save(DATA_PATH+"/stock_inputs_trimmed.npy",stock_inputs_trimmed)
np.save(DATA_PATH+"/stock_labels_trimmed.npy",stock_labels_trimmed)
np.save(DATA_PATH+"/stock_test_inputs_trimmed.npy",stock_test_inputs_trimmed)
np.save(DATA_PATH+"/stock_test_labels_trimmed.npy",stock_test_labels_trimmed)


In [ ]:
# ! cp  /content/idl-spring-22-project-deepar/data/stock_inputs_trimmed.npy  /content/drive/MyDrive/11785_experiments/single_company_data/
# ! cp  /content/idl-spring-22-project-deepar/data/stock_labels_trimmed.npy  /content/drive/MyDrive/11785_experiments/single_company_data/
# ! cp  /content/idl-spring-22-project-deepar/data/stock_test_inputs_trimmed.npy  /content/drive/MyDrive/11785_experiments/single_company_data/
# ! cp  /content/idl-spring-22-project-deepar/data/stock_test_labels_trimmed.npy  /content/drive/MyDrive/11785_experiments/single_company_data/

In [ ]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (500000, 192, 5)
stock_labels_trimmed shape (500000, 192)
stock_test_inputs_trimmed shape (100000, 192, 5)
stock_test_labels_trimmed shape (100000, 192)


In [ ]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])




In [ ]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

In [ ]:

def generate_horizon_weights(horizon_len):


  # weight = [i for i in range(1, horizon_len + 1)]
  
  weight = [1]
  for i in range(1, horizon_len):
    weight.append(weight[-1]*2)

  weight_arr = np.array(weight)

  return weight_arr/ np.sum(weight_arr)

HORIZON_PERIOD = 8

params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':64,
    'cov_dim': 3,
    'lstm_hidden_dim': 128,
    'lstm_layers':3 ,
    'window_size':192,
    'batch_size': 300,
    'learning_rate': 1e-3,
    'epochs':30,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'horizon_period': HORIZON_PERIOD,
    'horizon_weights': generate_horizon_weights(HORIZON_PERIOD)
}

# print(hor_weight)

assert sum(params['horizon_weights']) == 1.0
assert len(params['horizon_weights']) == params['horizon_period']

print(params)

{'num_classes': 242, 'embedding_dim': 64, 'cov_dim': 3, 'lstm_hidden_dim': 128, 'lstm_layers': 3, 'window_size': 192, 'batch_size': 300, 'learning_rate': 0.001, 'epochs': 30, 'num_test_samples': 100000, 'num_train_samples': 500000, 'conditioning_period': 168, 'prediction_period': 24, 'horizon_period': 8, 'horizon_weights': array([0.00392157, 0.00784314, 0.01568627, 0.03137255, 0.0627451 ,
       0.1254902 , 0.25098039, 0.50196078])}


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):

        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        
class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2
        

In [ ]:
class TrainDatasetHorizon(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]
        self.horizon_period = params['horizon_period']  # Return (t+1, t+2, t+3, ...., t+5) into the future


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):

        x = np.delete(self.data[index], 1, axis=1)
        x1 = torch.from_numpy(x.astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))

        x_shift = [None]*self.horizon_period

        x_shift = [torch.roll(x2, shifts=-1*i, dims=0)[:-1*(self.horizon_period - 1 )] for i in range(0, self.horizon_period)]

        # Retrun 0, 1, 2, 3, ......, num_samples - horizon_period samples
        # EG. Window Length of 192 (0-191), HORIZON Size of 5

        # IP Features will be from (0-187)

        # Note that the Original Dataset is already "1" shifted
        # There will be 5 Labels: 0-187 (1 shifted), 
        # 1-188 (2-shifted), 
        # 2-189 (3-shifted),
        # 3-190 (4-shifted),
        # 4-191 (5-shifted)
        
        return x1[:-1*(self.horizon_period - 1 ), :], x_shift

In [ ]:
# data_set = TrainDatasetHorizon()

# for x, x_shift in data_set:

#   print("x shape", x.shape, 'x shift', x_shift[0].shape)



In [ ]:
def unwindow_sequence(dataset, data_type="normal"):
  company_data = {}

  seq_id = 0
  company_index_cov_id = -1

  for cov_ip, cov_original_op in dataset:

    if data_type == "train_horizon":

      cov_op = cov_original_op[0]
      # print("---first cov op ---", cov_op)
      # print("---appending with 0----", cov_original_op[-1][-1*params['horizon_period']:])
      # cov_op = cov_op.append(cov_original_op[-1][-1*params['horizon_period']:])
      cov_op = torch.cat((cov_op, cov_original_op[-1][-1*(params['horizon_period'] - 1):]), dim = 0) 

      # print("cov op shape", cov_op.shape)
    else:
      cov_op = cov_original_op

    comp_index = cov_ip[seq_id][company_index_cov_id]
    company_name = index_to_company[comp_index.item()]

    # IP Shape (Time, Dim)
    # OP Shape (Time, 1) # One covariate (open price) per time instant
    if company_data.get(company_name, None) is None:
      company_data[company_name] = [cov_ip, cov_op]
      
    else:
      # windows are created with stride 1
      # so we should be appending only the last element in the sequence
      cov_ip_last = cov_ip[-1, :].unsqueeze(0)
      cov_op_last = cov_op[-1].unsqueeze(0)

      # 0 --> Covariates
      # 1 --> Outputs, aka stock price
      company_data[company_name][0] = torch.cat((company_data[company_name][0], cov_ip_last), dim=0)
      company_data[company_name][1] = torch.cat((company_data[company_name][1], cov_op_last), dim=0)

  return company_data

In [ ]:
train_data = TrainDatasetHorizon()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False)

# train_data_unwindowed = unwindow_sequence(train_data, data_type="train_horizon")
# test_data_unwindowed = unwindow_sequence(test_data)


In [ ]:
# print("train_data_unwindowed", train_data_unwindowed.shape)
# print("test_data_unwindowed", test_data_unwindowed.shape)

In [ ]:
print(train_data_unwindowed.keys())
print(len(train_data_unwindowed))

NameError: ignored

In [ ]:
print(test_data_unwindowed.keys())
print(len(test_data_unwindowed))
# Some Companies are there in the Train Set, But not in the Test Set. 
# However, all companies in the Test Set is there in the Train Set

NameError: ignored

In [ ]:
import plotly.express as px

def plot_output(company_data, company_name):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[company_name][1]), num=len(company_data[company_name][1]))
  fig = px.line(x=x, y=company_data[company_name][1])
  fig.show()
  print("op", company_data[company_name][1])

def plot_input_covariate(company_data, company_name, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[company_name][0].shape[0], num=len(company_data[company_name][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[company_name][0][:,covariate_index])
  fig.show()
  print("ip", company_data[company_name][0][:,covariate_index])

In [ ]:
# SANITY Check One: Output Label (open price) is shifted by one element of input (the ip opening price)
plot_output(train_data_unwindowed, 'MCD')
plot_input_covariate(train_data_unwindowed, 'MCD', 0)

op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4594,  0.4971,  0.4838])


In [ ]:

# SANITY Check TWO: Train and Test data are contiguous
plot_input_covariate(train_data_unwindowed, 'MCD', 0)
# plot_input_covariate(test_data_unwindowed, 'MCD', 0)
plot_output(train_data_unwindowed, 'MCD')

ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4591,  0.4498,  0.4591])


op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


In [ ]:
# SANITY Check TWO for Output Labels: Train and Test data are contiguous

plot_output(train_data_unwindowed, 'MCD')
plot_output(test_data_unwindowed, 'MCD')

op tensor([-0.6790, -0.7051, -0.6279,  ...,  0.4282,  0.4346,  0.4358])


op tensor([0.4304, 0.4237, 0.4167,  ..., 1.7638, 1.7802, 1.7692])


In [ ]:
plot_input_covariate(train_data_unwindowed, 'MCD', 0)  # Opening Price
# plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Volume
plot_input_covariate(train_data_unwindowed, 'MCD', 1)  # Day
plot_input_covariate(train_data_unwindowed, 'MCD', 2)  # Month
plot_input_covariate(train_data_unwindowed, 'MCD', 3)  # Company ID


ip tensor([-0.6889, -0.6790, -0.7051,  ...,  0.4594,  0.4971,  0.4838])


ip tensor([-1.6811, -1.5670, -1.4528,  ...,  0.3735,  0.4877,  0.6018])


ip tensor([ 1.2869,  1.2869,  1.2869,  ..., -0.4512, -0.4512, -0.4512])


ip tensor([155., 155., 155.,  ..., 155., 155., 155.])


In [ ]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])


Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ff204d73410>> (for pre_run_cell):


Exception: ignored

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7ff204d73410>> (for post_run_cell):


Exception: ignored

In [ ]:
plot_output_windowed(test_data, 0)
plot_input_covariate_windowed(test_data, 0, 0)

op tensor([0.8988, 0.6912, 0.8988, 0.8148, 0.9591, 0.8196, 0.8465, 0.8114, 0.9247,
        0.8378, 0.8026, 0.7849, 0.7449, 0.8290, 0.9376, 0.7982, 0.6003, 0.5079,
        0.6957, 0.8114, 1.3751, 1.4421, 1.6040, 1.7501, 1.6682, 1.6078, 1.6757,
        1.5697, 1.4694, 1.6154, 1.7538, 1.5198, 1.8649, 2.5644, 1.4421, 1.3313,
        1.6229, 1.1855, 1.4107, 1.2183, 1.1484, 0.8070, 0.4564, 0.9676, 0.8988,
        0.9376, 1.0398, 1.1691, 1.2040, 1.1608, 0.8727, 0.9847, 0.8944, 0.9376,
        1.2912, 1.1896, 0.7494, 1.0187, 0.6686, 0.5682, 0.6957, 0.7271, 1.1896,
        1.1773, 1.0356, 1.0651, 1.2019, 1.3513, 1.0861, 1.1649, 1.1111, 0.8070,
        0.7449, 0.5405, 0.4299, 0.6912, 0.4611, 0.6957, 1.1194, 1.2015, 1.3632,
        1.3233, 1.3313, 1.2019, 1.1773, 1.0017, 0.8944, 0.8814, 0.4846, 0.5266,
        0.5405, 0.4799, 0.6049, 0.6414, 0.5219, 0.7316, 1.0861, 1.0567, 0.9591,
        0.9161, 0.9161, 0.8596, 0.9419, 0.9591, 0.9505, 0.9161, 0.9591, 0.9376,
        1.2101, 1.2101, 1.0735, 1.044

ip tensor([0.7938, 0.8988, 0.6912, 0.8988, 0.8148, 0.9591, 0.8196, 0.8465, 0.8114,
        0.9247, 0.8378, 0.8026, 0.7849, 0.7449, 0.8290, 0.9376, 0.7982, 0.6003,
        0.5079, 0.6957, 0.8114, 1.3751, 1.4421, 1.6040, 1.7501, 1.6682, 1.6078,
        1.6757, 1.5697, 1.4694, 1.6154, 1.7538, 1.5198, 1.8649, 2.5644, 1.4421,
        1.3313, 1.6229, 1.1855, 1.4107, 1.2183, 1.1484, 0.8070, 0.4564, 0.9676,
        0.8988, 0.9376, 1.0398, 1.1691, 1.2040, 1.1608, 0.8727, 0.9847, 0.8944,
        0.9376, 1.2912, 1.1896, 0.7494, 1.0187, 0.6686, 0.5682, 0.6957, 0.7271,
        1.1896, 1.1773, 1.0356, 1.0651, 1.2019, 1.3513, 1.0861, 1.1649, 1.1111,
        0.8070, 0.7449, 0.5405, 0.4299, 0.6912, 0.4611, 0.6957, 1.1194, 1.2015,
        1.3632, 1.3233, 1.3313, 1.2019, 1.1773, 1.0017, 0.8944, 0.8814, 0.4846,
        0.5266, 0.5405, 0.4799, 0.6049, 0.6414, 0.5219, 0.7316, 1.0861, 1.0567,
        0.9591, 0.9161, 0.9161, 0.8596, 0.9419, 0.9591, 0.9505, 0.9161, 0.9591,
        0.9376, 1.2101, 1.2101, 1.073

In [ ]:
print('train data shape', train_data[0][0].shape)

train data shape torch.Size([179, 4])


# MODEL


In [ ]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim'],
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )


        self.distribution_premu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_mu = nn.GELU()
        self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_sigma = nn.Softplus()


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        onehot_embed = self.embedding(company_index)
        
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]

        assert cov_dim == params['cov_dim']
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_premu = self.distribution_premu(out1)
        out_mu = self.distribution_mu(out_premu)

        out_presigma = self.distribution_presigma(out1)
        out_sigma = self.distribution_sigma(out_presigma)

        return out_mu, out_sigma, hn_cn



In [ ]:
model = Network(params)

In [ ]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([300, 185, 1])
out_sigma shape torch.Size([300, 185, 1])
ht shape torch.Size([3, 300, 128]) torch.Size([3, 300, 128])


In [ ]:
def loss_fn(mu,sigma,labels):

  batch,seq_len,mudim = mu.shape
  batch,seq_len,sigmadim = sigma.shape

  assert mudim==sigmadim

  mu = mu.view(batch*seq_len,mudim)
  sigma = sigma.view(batch*seq_len,sigmadim)
  labels = labels.view(batch*seq_len,1)

  gauss_prices = torch.distributions.Normal(mu,sigma)
  total_likelihood = -1 * torch.sum(gauss_prices.log_prob(labels))/(batch*seq_len)

  return total_likelihood

In [ ]:
def accuracy_RMSE_train(mu: torch.Tensor, labels: torch.Tensor):
  predictions = mu.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [ ]:
def accuracy_RMSE_test(predictions: torch.Tensor, labels: torch.Tensor):
  predictions = predictions.cpu().detach().numpy()
  labels = labels.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sqrt((1/(N)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, _ = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result


In [ ]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [ ]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
        
  rmse_val = rmse(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])
  nd_val = ND(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])

  # print("overall mu", overall_mu[:, params['conditioning_period']:].shape)
  # print("overall label", overall_label.shape)

  return rmse_val, nd_val







In [ ]:
# rmse = accuracy_RMSE_train(overall_mu, overall_label)
# print("rmse ", rmse)

In [ ]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])


In [ ]:
validate(model)

(0.2377735869947597, 0.9873128)

In [ ]:
!pip install wandb
!wandb login

wandb: Currently logged in as: mmml (use `wandb login --relogin` to force relogin)


In [ ]:
import wandb

wandb.init(project="final-results", entity="idl-deepar-spring-22", name="single_weighted_horizon&8_exponent&weight_hiddim128_&loss_no&volume")

wandb: Currently logged in as: mmml (use `wandb login --relogin` to force relogin)


In [ ]:
wandb.finish()

ND,█▁▄
loss,█▁▆
rmse,█▁▅
ND,0.51227
loss,-0.14216
rmse,0.15476


# TRAIN

In [ ]:
model_version='single_weighted_horizon&8_hiddim128_exponent&weight_&loss_no&volume.pt'
epochs = 24

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)

print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0

    if os.path.exists(f'/content/drive/MyDrive/DeepARExperiments/{model_version}'):
        # model.load_state_dict(torch.load(f'{SAVE_PATH}{EXP_TAG}/model_saved_epoch{epoch-1}.pt')) 

        checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch'] + 1

    for i, (ip_covariate, op_label_list) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        op_label_list     = [op_label.cuda() for op_label in op_label_list]

        mu, sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        # loss = [loss_fn(mu, sigma, op_label) * params['horizon_weights'] for op_label in op_label_list]

        loss = 0
        for i, op_label in enumerate(op_label_list):
          loss +=  loss_fn(mu, sigma, op_label) * params['horizon_weights'][i]

        total_loss += float(loss)
        loss.backward()
        optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse, val_nd= validate(model=model)
    scheduler.step(val_rmse)

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/DeepARExperiments/{model_version}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
      torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'val_rmse': val_rmse,
              'loss': total_loss/len(train_loader),
              },  f'/content/drive/MyDrive/DeepARExperiments/best_dev_acc_{best_dev_rmse}_{model_version}')
    
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse, "nd is", val_nd)
    wandb.log({"loss":loss, "rmse": val_rmse, "ND": val_nd})

Network(
  (embedding): Embedding(242, 64)
  (lstm): LSTM(67, 128, num_layers=3, batch_first=True)
  (distribution_premu): Linear(in_features=128, out_features=1, bias=True)
  (distribution_mu): GELU()
  (distribution_presigma): Linear(in_features=128, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


Epoch 5/24: Train Loss -0.4843, Learning Rate 0.0009
rmse is  0.1515047861638465 nd is 0.5021194


Epoch 6/24: Train Loss -0.4902, Learning Rate 0.0009
rmse is  0.15156725489689138 nd is 0.49915853


Epoch 7/24: Train Loss -0.5039, Learning Rate 0.0009
rmse is  0.15194576794548822 nd is 0.50039715


Epoch 8/24: Train Loss -0.5120, Learning Rate 0.0009
rmse is  0.15123894060192333 nd is 0.49727076


...

Epoch 24/24: Train Loss -0.5897, Learning Rate 0.0004
rmse is  0.15156599420089908 nd is 0.49381778


Epoch 25/24: Train Loss -0.5916, Learning Rate 0.0004
rmse is  0.15103394000352335 nd is 0.4916948


Epoch 26/24: Train Loss -0.5911, Learning Rate 0.0004
rmse is  0.15009018175685726 nd is 0.48879746


Epoch 27/24: Train Loss -0.5909, Learning Rate 0.0003
rmse is  0.1505004974648192 nd is 0.48937657


Epoch 28/24: Train Loss -0.5993, Learning Rate 0.0003
rmse is  0.15115313436635913 nd is 0.4916692


In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/best_dev_acc_0.030142039248817844_deepar_model_no_volume_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:

def get_pred_sigma_mu_labels(model):

  model.eval()
  model.cuda()


  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -1, :].unsqueeze(1), sigma[:, -1, :].unsqueeze(1)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        
        pred_mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        batch_mu = torch.cat((batch_mu,  pred_mu.squeeze(2)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)

  return overall_mu, overall_sigma, overall_label

In [ ]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, sigma, window_id):
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [ ]:
# #s for the paper 3500 420
# model_no_volume = Network(params) 
# checkpoint = torch.load('/content/drive/MyDrive/DeepARExperiments/deepar_model_no_volume_v1.pt')
# model_no_volume.load_state_dict(checkpoint['model_state_dict'])

model_no_volume = model

In [ ]:
# mu_no_volume, sigma_no_volume, label_no_volume =  get_pred_sigma_mu_labels(model)

# 420
# 5000
# 7000
# 11000
# 9900
# 9613
# 7105
# 2541
# 9632
# 63421
# 52132
# 57031
# 58031
# 67031
# 7031
# 9921
# 69000

plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 69000)

In [ ]:
rmse = accuracy_RMSE_test(mu_no_volume[:, 168:], label_no_volume[:, 168:])
print('rmse ', rmse)

rmse  0.14072188527960527


In [ ]:
# #s for the paper 3500 420 2500 69

# for i in range(0, 3500, 70):
plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 2550  )

In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result

print("rmse ", rmse(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

print("nd ", ND(mu_no_volume[2500:2501,168:], label_no_volume[2500:2501,168:]))

rmse  0.06767744374120029
nd  0.057781804
